In [ ]:
# === CONFIGURAÇÃO DA API OPENROUTER (OPCIONAL) ===
# Monitor LLM para otimização inteligente do treinamento

import os
import getpass

# Configuração da API OpenRouter
ENABLE_LLM_MONITOR = True # @param {type:"boolean"}

if ENABLE_LLM_MONITOR:
    print("🤖 Configurando Monitor LLM com OpenRouter...")
    print("📋 O monitor LLM analisa o progresso do treinamento e sugere ajustes automáticos")
    print("💡 Opcional: deixe em branco para desabilitar")
    
    # Input da API key (campo seguro)
    OPENROUTER_API_KEY = getpass.getpass("🔑 Digite sua API Key do OpenRouter (ou Enter para pular): ")
    
    if OPENROUTER_API_KEY and OPENROUTER_API_KEY.strip():
        # Configurar variável de ambiente
        os.environ['OPENROUTER_API_KEY'] = OPENROUTER_API_KEY.strip()
        
        # Testar conexão
        try:
            import requests
            test_url = "https://openrouter.ai/api/v1/models"
            headers = {
                "Authorization": f"Bearer {OPENROUTER_API_KEY.strip()}",
                "Content-Type": "application/json"
            }
            
            response = requests.get(test_url, headers=headers, timeout=10)
            if response.status_code == 200:
                print("✅ API Key válida - Monitor LLM habilitado")
                print("📊 Funcionalidades ativas:")
                print("   - Análise automática a cada 10 épocas")
                print("   - Sugestões de ajuste de learning rate")
                print("   - Detecção de problemas de convergência")
                print("   - Relatórios detalhados de progresso")
                
                # Mostrar modelos disponíveis
                models = response.json().get('data', [])
                claude_models = [m for m in models if 'claude' in m.get('id', '').lower()]
                if claude_models:
                    print(f"🧠 Modelo recomendado: {claude_models[0].get('id', 'claude-3-5-sonnet')}")
            else:
                print(f"⚠️ Erro na validação da API: {response.status_code}")
                print("🔧 Monitor LLM será desabilitado para este treinamento")
                ENABLE_LLM_MONITOR = False
                
        except Exception as e:
            print(f"⚠️ Erro ao testar API: {e}")
            print("🔧 Monitor LLM será desabilitado para este treinamento")
            ENABLE_LLM_MONITOR = False
    else:
        print("ℹ️ API Key não fornecida - Monitor LLM desabilitado")
        ENABLE_LLM_MONITOR = False
else:
    print("🔧 Monitor LLM desabilitado por configuração")
    ENABLE_LLM_MONITOR = False

# Salvar configuração para uso posterior
LLM_CONFIG = {
    'enabled': ENABLE_LLM_MONITOR,
    'provider': 'openrouter',
    'model': 'anthropic/claude-3-5-sonnet-20241022',
    'monitor_every_epochs': 10,
    'api_key_configured': bool(os.environ.get('OPENROUTER_API_KEY'))
}

print(f"\n🎯 Configuração LLM: {'✅ Ativo' if ENABLE_LLM_MONITOR else '❌ Desabilitado'}")

if not ENABLE_LLM_MONITOR:
    print("\n💡 Para habilitar o Monitor LLM:")
    print("1. Crie uma conta em https://openrouter.ai")
    print("2. Obtenha sua API Key")
    print("3. Execute esta célula novamente e forneça a key")
    print("4. O monitor ajudará a otimizar seu treinamento automaticamente")

In [ ]:
# === CONFIGURAÇÃO DO MODO DE TREINAMENTO ===
# Configuração otimizada para A100 40GB VRAM
DEBUG_MODE = False # @param {type:"boolean"}
USE_DRIVE = True # @param {type:"boolean"}
MOUNT_DRIVE = True # @param {type:"boolean"}

# Detectar GPU automaticamente e ajustar configuração
import subprocess
def get_gpu_info():
    try:
        result = subprocess.run(['nvidia-smi', '--query-gpu=name,memory.total', '--format=csv,noheader,nounits'], 
                              capture_output=True, text=True)
        if result.returncode == 0:
            gpu_info = result.stdout.strip().split('\n')[0]
            gpu_name, gpu_memory = gpu_info.split(', ')
            return gpu_name.strip(), int(gpu_memory.strip())
    except:
        pass
    return "Unknown", 0

gpu_name, gpu_memory = get_gpu_info()
print(f"🎮 GPU Detectada: {gpu_name}")
print(f"💾 VRAM: {gpu_memory} MB")

# Configuração automática baseada na GPU
if "A100" in gpu_name and gpu_memory >= 40000:
    print("🚀 MODO A100: Configuração de alta performance")
    BATCH_SIZE = 32
    CONFIG_NAME = "vits2_english_a100_optimized"
    EXPECTED_PERFORMANCE = "5-8 it/s"
    ESTIMATED_TIME = "4-6 horas"
elif "V100" in gpu_name:
    print("⚡ MODO V100: Configuração otimizada")
    BATCH_SIZE = 24
    CONFIG_NAME = "vits2_english_production"
    EXPECTED_PERFORMANCE = "3-5 it/s"
    ESTIMATED_TIME = "6-8 horas"
elif "T4" in gpu_name:
    print("📱 MODO T4: Configuração conservativa")
    BATCH_SIZE = 16
    CONFIG_NAME = "vits2_english_production"
    EXPECTED_PERFORMANCE = "1-2 it/s"
    ESTIMATED_TIME = "12-15 horas"
else:
    print("⚠️ GPU não detectada, usando configuração padrão")
    BATCH_SIZE = 16
    CONFIG_NAME = "vits2_english_production"
    EXPECTED_PERFORMANCE = "1-2 it/s"
    ESTIMATED_TIME = "12-15 horas"

# Configuração do treinamento
if DEBUG_MODE:
    print("🐛 MODO DEBUG: Treinamento rápido para teste")
    EPOCHS = 3
    MAX_SAMPLES = 100
    CONFIG_NAME = "vits2_english_debug"
    BATCH_SIZE = min(BATCH_SIZE, 8)  # Reduzido para debug
else:
    print("🚀 MODO PRODUÇÃO: Treinamento completo")
    EPOCHS = 200
    MAX_SAMPLES = None

print(f"📊 Configuração: {CONFIG_NAME}")
print(f"🔄 Épocas: {EPOCHS}")
print(f"📦 Batch Size: {BATCH_SIZE}")
print(f"📈 Amostras: {MAX_SAMPLES if MAX_SAMPLES else 'Todas (22.910)'}")
print(f"⚡ Performance esperada: {EXPECTED_PERFORMANCE}")
print(f"⏱️ Tempo estimado: {ESTIMATED_TIME}")

In [ ]:
# === CONFIGURAÇÃO DO SISTEMA E CLONAGEM ===
import subprocess
import sys
import os

def run_command(cmd, description):
    """Executa comando com output em tempo real."""
    print(f"🔄 {description}")
    process = subprocess.Popen(
        cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT,
        universal_newlines=True, bufsize=1
    )

    for line in process.stdout:
        print(line.rstrip())

    process.wait()
    if process.returncode != 0:
        raise RuntimeError(f"Comando falhou: {cmd}")
    print(f"✅ {description} - Concluído\n")

# Montar Google Drive se necessário
if MOUNT_DRIVE:
    from google.colab import drive
    drive.mount('/content/drive')

    if USE_DRIVE:
        drive_path = '/content/drive/MyDrive/ValeTTS'
        os.makedirs(drive_path, exist_ok=True)
        os.chdir(drive_path)
        print(f"📁 Diretório de trabalho: {drive_path}")

# Verificar GPU
run_command("nvidia-smi", "Verificando GPU disponível")

# Clonar repositório
if not os.path.exists('ValeTTS'):
    run_command(
        "git clone https://github.com/wallaceblaia/ValeTTS-Colab.git ValeTTS",
        "Clonando repositório ValeTTS"
    )
else:
    print("📁 Repositório já existe")

os.chdir('ValeTTS')
run_command("git pull origin main", "Atualizando repositório")
print(f"📍 Diretório atual: {os.getcwd()}")


In [ ]:
# === INSTALAÇÃO DE DEPENDÊNCIAS ESPECÍFICAS PARA INGLÊS ===

print("🔧 Instalando dependências do sistema para inglês...")

# Instalar dependências do sistema para inglês
system_deps = [
    "apt-get update",
    "apt-get install -y espeak espeak-data libespeak1 libespeak-dev",
    "apt-get install -y ffmpeg sox libsox-fmt-all",
    "apt-get install -y language-pack-en",  # Suporte específico para inglês
]

for cmd in system_deps:
    run_command(cmd, f"Sistema: {cmd.split()[-1]}")

print("📦 Instalando dependências Python...")

# Lista de dependências Python essenciais
essential_deps = [
    "pip install phonemizer==3.3.0",  # G2P essencial
    "pip install num2words",          # Conversão números para inglês
    "pip install inflect",            # Pluralização e ordinais em inglês
]

# Lista de dependências opcionais
optional_deps = [
    ("pip install eSpeak", "eSpeak Python interface"),
    ("pip install espeak-ng", "eSpeak-NG alternativa"),
]

# Instalar dependências essenciais
for cmd in essential_deps:
    run_command(cmd, f"Python essencial: {cmd.split()[-1]}")

# Tentar instalar dependências opcionais
for cmd, desc in optional_deps:
    try:
        run_command(cmd, f"Python opcional: {desc}")
        break  # Se uma funcionar, não precisamos das outras
    except RuntimeError:
        print(f"⚠️ {desc} não disponível, tentando próxima...")

print("🔧 Instalando ValeTTS...")
# Instalar ValeTTS em modo desenvolvimento
run_command("pip install -e .", "Instalando ValeTTS em modo desenvolvimento")

# Verificar instalações críticas
print("🧪 Verificando instalações...")

# Verificações essenciais
essential_checks = [
    ("python -c 'import phonemizer; print(f\"✅ Phonemizer: {phonemizer.__version__}\")'", "Phonemizer"),
    ("python -c 'import valetts; print(f\"✅ ValeTTS instalado\")'", "ValeTTS"),
]

# Verificações opcionais
optional_checks = [
    ("espeak --version", "ESpeak sistema"),
    ("python -c 'from valetts.data.preprocessing.text_en import EnglishTextPreprocessor; print(\"✅ EnglishTextPreprocessor disponível\")'", "Processador inglês"),
]

# Executar verificações essenciais
for cmd, desc in essential_checks:
    run_command(cmd, f"Verificar {desc}")

# Executar verificações opcionais
for cmd, desc in optional_checks:
    try:
        run_command(cmd, f"Verificar {desc}")
    except Exception as e:
        print(f"⚠️ Aviso: {desc} - {e}")

# Teste robusto do processador de inglês
print("🧪 Testando processamento de inglês...")

test_code_robust = '''
import sys
import os
import warnings
warnings.filterwarnings("ignore")

print("🔍 Testando importações...")

# Teste 1: Importar ValeTTS
try:
    import valetts
    print("✅ ValeTTS importado")
except Exception as e:
    print(f"❌ ValeTTS: {e}")
    sys.exit(1)

# Teste 2: Importar processador de texto inglês
try:
    from valetts.data.preprocessing.text_en import EnglishTextPreprocessor
    print("✅ EnglishTextPreprocessor importado")
except Exception as e:
    print(f"❌ EnglishTextPreprocessor: {e}")
    sys.exit(1)

# Teste 3: Criar processador (sem phonemes primeiro)
try:
    processor_basic = EnglishTextPreprocessor(
        language="en-us",
        use_phonemes=False,  # Desabilitar phonemes para teste básico
        normalize_numbers=True
    )
    print("✅ Processador básico criado")
except Exception as e:
    print(f"❌ Processador básico: {e}")
    sys.exit(1)

# Teste 4: Normalização básica
try:
    test_text = "Hello world! This is a test with numbers 123 and Mr. Smith."
    normalized = processor_basic.normalize_text(test_text)
    print(f"✅ Texto normalizado: {normalized}")
except Exception as e:
    print(f"❌ Normalização: {e}")

# Teste 5: Processador com phonemes (opcional)
try:
    processor_phonemes = EnglishTextPreprocessor(
        language="en-us",
        use_phonemes=True,
        normalize_numbers=True
    )
    print("✅ Processador com phonemes criado")
    
    # Teste phonemes
    try:
        phonemes = processor_phonemes.text_to_phonemes("hello world")
        print(f"✅ Phonemes: {phonemes}")
    except Exception as e:
        print(f"⚠️ Phonemes: {e} - Funcionalidade opcional")
        
except Exception as e:
    print(f"⚠️ Processador com phonemes: {e} - Funcionalidade opcional")

print("\\n✅ Teste básico do processador concluído!")
print("💡 O sistema está pronto para treinamento mesmo se phonemes falharam")
'''

# Executar teste robusto
try:
    with open("test_english_robust.py", "w") as f:
        f.write(test_code_robust)
    run_command("python test_english_robust.py", "Teste robusto do processador inglês")
except Exception as e:
    print(f"❌ Teste falhou: {e}")
    print("💡 Pode haver problemas com as dependências")
finally:
    # Limpar arquivo de teste
    if os.path.exists("test_english_robust.py"):
        os.remove("test_english_robust.py")

print("\n" + "="*60)
print("✅ INSTALAÇÃO CONCLUÍDA!")
print("="*60)
print("🎯 Sistema configurado para treinamento VITS2 em inglês")
print("")
print("📋 Componentes instalados:")
print("   ✅ ESpeak (sistema) - para G2P em inglês")
print("   ✅ Phonemizer 3.3.0 - conversão texto→phonema")
print("   ✅ num2words - números em inglês")
print("   ✅ inflect - pluralização em inglês")
print("   ✅ ValeTTS - framework principal")
print("   ✅ EnglishTextPreprocessor - processamento específico")
print("")
print("💡 NOTAS IMPORTANTES:")
print("   - Se phonemes falharam, o treinamento ainda funciona")
print("   - O sistema usará fallback para processamento de texto")
print("   - A performance pode ser ligeiramente reduzida mas funcional")
print("")
print("🚀 PRÓXIMO PASSO: Execute a célula de download do dataset!")

# Verificação final dos imports críticos
print("\n🔍 Verificação final...")
try:
    import valetts
    from valetts.data.preprocessing.text_en import EnglishTextPreprocessor
    from valetts.models.vits2.model import VITS2
    from valetts.models.vits2.config import VITS2Config
    print("✅ Todos os imports críticos funcionando!")
except Exception as e:
    print(f"❌ Problema crítico: {e}")
    print("🔄 Pode ser necessário reiniciar o runtime e tentar novamente")

In [ ]:
# === DOWNLOAD E VERIFICAÇÃO COMPLETA DO DATASET ===
import os
import pandas as pd
from pathlib import Path

print("📥 INICIANDO VERIFICAÇÃO COMPLETA DO DATASET...")
print("="*60)

# Configuração de caminhos
dataset_path = "data/generated/Dataset-Unificado"
metadata_file = f"{dataset_path}/metadata.csv"
audio_dir = f"{dataset_path}/audio"
audio_raw_dir = f"{dataset_path}/audio/raw"

print(f"📁 Caminhos configurados:")
print(f"   Dataset: {dataset_path}")
print(f"   Metadata: {metadata_file}")
print(f"   Áudio: {audio_raw_dir}")

# ETAPA 1: Verificar se dataset existe localmente
print(f"\n🔍 ETAPA 1: Verificando existência local...")
if os.path.exists(metadata_file):
    print("✅ Dataset encontrado localmente")
    print(f"   📄 Metadata: {metadata_file}")
    print(f"   📊 Tamanho: {os.path.getsize(metadata_file):,} bytes")
else:
    print("📥 Dataset não encontrado localmente")
    
    # ETAPA 2: Verificar Google Drive
    print(f"\n🔍 ETAPA 2: Verificando Google Drive...")
    drive_dataset = "/content/drive/MyDrive/ValeTTS/data/generated/Dataset-Unificado"
    
    if os.path.exists(drive_dataset):
        print(f"✅ Dataset encontrado no Drive!")
        print(f"   📁 Local: {drive_dataset}")
        
        # Mostrar conteúdo do Drive
        drive_contents = os.listdir(drive_dataset)
        print(f"   📋 Conteúdo: {drive_contents}")
        
        print(f"\n🔗 Criando estrutura local...")
        os.makedirs("data/generated", exist_ok=True)
        
        # Verificar se já existe link/diretório
        if os.path.exists(dataset_path):
            if os.path.islink(dataset_path):
                print("   🗑️ Removendo link simbólico anterior...")
                os.unlink(dataset_path)
            elif os.path.isdir(dataset_path):
                print("   ⚠️ Diretório já existe, usando existente")
        
        if not os.path.exists(dataset_path):
            print("   🔗 Criando link simbólico...")
            os.symlink(drive_dataset, dataset_path)
            print("   ✅ Link criado com sucesso!")
    else:
        print("❌ Dataset não encontrado no Drive")
        print(f"   📁 Procurado em: {drive_dataset}")
        
        print(f"\n💡 SOLUÇÕES PARA OBTER O DATASET:")
        print(f"")
        print(f"🔧 MÉTODO 1 - Google Drive (RECOMENDADO):")
        print(f"   1. Faça upload do dataset para:")
        print(f"      {drive_dataset}")
        print(f"   2. Execute esta célula novamente")
        print(f"")
        print(f"🔧 MÉTODO 2 - Upload direto:")
        print(f"   1. Use o ícone 📁 no painel esquerdo")
        print(f"   2. Carregue para: {dataset_path}")
        print(f"")
        print(f"🔧 MÉTODO 3 - Download automático:")
        print(f"   Substitua SUA_URL pela URL do Google Drive:")
        print(f"   !gdown --folder 'SUA_URL' -O {dataset_path}")
        
        # Parar aqui se não encontrou o dataset
        print(f"\n⛔ PARANDO: Dataset não encontrado!")
        print(f"📋 Execute uma das soluções acima primeiro.")

# ETAPA 3: Verificação detalhada do dataset
if os.path.exists(metadata_file):
    print(f"\n🔍 ETAPA 3: Analisando estrutura do dataset...")
    
    # Verificar estrutura de diretórios
    print(f"📁 Verificando estrutura:")
    structure_checks = [
        (dataset_path, "Dataset base"),
        (audio_dir, "Pasta audio"),
        (audio_raw_dir, "Pasta audio/raw"),
        (metadata_file, "Arquivo metadata.csv")
    ]
    
    for path, description in structure_checks:
        exists = os.path.exists(path)
        if exists:
            if os.path.isfile(path):
                size = os.path.getsize(path)
                print(f"   ✅ {description}: {size:,} bytes")
            else:
                items = len(os.listdir(path))
                print(f"   ✅ {description}: {items:,} itens")
        else:
            print(f"   ❌ {description}: NÃO ENCONTRADO")
    
    # ETAPA 4: Análise do metadata.csv
    print(f"\n🔍 ETAPA 4: Carregando e analisando metadata...")
    try:
        print(f"📊 Carregando CSV...")
        df = pd.read_csv(metadata_file)
        
        total_samples = len(df)
        unique_speakers = df['speaker_id'].nunique()
        locales = df['locale'].unique() if 'locale' in df.columns else ['unknown']
        
        print(f"✅ Metadata carregado com sucesso!")
        print(f"   📈 Total de amostras: {total_samples:,}")
        print(f"   🎤 Falantes únicos: {unique_speakers}")
        print(f"   🌍 Idiomas: {locales}")
        print(f"   📋 Colunas: {list(df.columns)}")
        
        # Verificar idioma inglês
        if 'locale' in df.columns:
            english_samples = len(df[df['locale'] == 'en']) if 'en' in locales else 0
            print(f"   🇺🇸 Amostras em inglês: {english_samples:,}")
            
            if english_samples == 0:
                print("   ❌ PROBLEMA: Nenhuma amostra em inglês!")
            elif english_samples < 1000:
                print("   ⚠️ AVISO: Poucas amostras para treinamento robusto")
            else:
                print("   ✅ Quantidade adequada para treinamento")
        
        # Verificar colunas obrigatórias
        required_columns = ['audio_path', 'text_normalized', 'speaker_id']
        missing_columns = [col for col in required_columns if col not in df.columns]
        
        if missing_columns:
            print(f"   ❌ PROBLEMA: Colunas ausentes: {missing_columns}")
        else:
            print(f"   ✅ Todas as colunas obrigatórias presentes")
        
        # ETAPA 5: Verificação dos arquivos de áudio
        print(f"\n🔍 ETAPA 5: Verificando arquivos de áudio...")
        
        # Contar arquivos reais
        if os.path.exists(audio_raw_dir):
            wav_files = [f for f in os.listdir(audio_raw_dir) if f.endswith('.wav')]
            total_wav_files = len(wav_files)
            print(f"📊 Arquivos WAV encontrados: {total_wav_files:,}")
            
            # Verificar correspondência
            csv_entries = len(df)
            if total_wav_files == csv_entries:
                print(f"✅ PERFEITO: Número de arquivos corresponde ao CSV!")
            elif total_wav_files == csv_entries - 1:  # Considerando header
                print(f"✅ BOM: Arquivos correspondem (considerando header do CSV)")
            else:
                print(f"⚠️ DIVERGÊNCIA: CSV={csv_entries}, WAV={total_wav_files}")
            
            # Verificar primeiros arquivos
            print(f"\n🔍 Testando primeiros 10 arquivos...")
            sample_size = min(10, len(df))
            found_count = 0
            
            for i in range(sample_size):
                audio_path_csv = df.iloc[i]['audio_path']
                full_path = os.path.join(dataset_path, audio_path_csv)
                
                if os.path.exists(full_path):
                    size = os.path.getsize(full_path)
                    found_count += 1
                    print(f"   ✅ {audio_path_csv} ({size:,} bytes)")
                else:
                    print(f"   ❌ {audio_path_csv} - NÃO ENCONTRADO")
            
            print(f"\n📊 Resultado da amostra: {found_count}/{sample_size} arquivos encontrados")
            
            # Verificar tamanhos dos arquivos
            if found_count > 0:
                print(f"\n📊 Estatísticas dos arquivos:")
                sizes = []
                for i in range(min(100, len(df))):  # Amostra de 100 arquivos
                    audio_path_csv = df.iloc[i]['audio_path']
                    full_path = os.path.join(dataset_path, audio_path_csv)
                    if os.path.exists(full_path):
                        sizes.append(os.path.getsize(full_path))
                
                if sizes:
                    avg_size = sum(sizes) / len(sizes)
                    min_size = min(sizes)
                    max_size = max(sizes)
                    print(f"   📏 Tamanho médio: {avg_size:,.0f} bytes")
                    print(f"   📏 Menor arquivo: {min_size:,} bytes")
                    print(f"   📏 Maior arquivo: {max_size:,} bytes")
            
            # Verificar formato dos arquivos
            print(f"\n🔍 Verificando formato do primeiro arquivo...")
            if found_count > 0:
                first_audio = os.path.join(dataset_path, df.iloc[0]['audio_path'])
                import subprocess
                try:
                    result = subprocess.run(['file', first_audio], capture_output=True, text=True)
                    print(f"   🎵 Formato: {result.stdout.strip()}")
                except:
                    print(f"   ⚠️ Não foi possível verificar o formato")
        
        else:
            print(f"❌ Pasta de áudio não encontrada: {audio_raw_dir}")
        
        # ETAPA 6: Resumo final
        print(f"\n" + "="*60)
        print(f"📊 RESUMO FINAL DO DATASET:")
        print(f"="*60)
        print(f"✅ Metadata: {total_samples:,} amostras")
        print(f"✅ Falantes: {unique_speakers}")
        print(f"✅ Idioma: inglês ({english_samples:,} amostras)")
        print(f"✅ Arquivos WAV: {total_wav_files:,}")
        print(f"✅ Correspondência: {found_count}/{sample_size} testados")
        
        # Status final
        all_good = (
            english_samples > 0 and 
            not missing_columns and 
            total_wav_files > 0 and 
            found_count == sample_size
        )
        
        if all_good:
            print(f"\n🚀 DATASET COMPLETAMENTE PRONTO!")
            print(f"✅ Pode prosseguir para o treinamento")
            print(f"📊 Performance esperada: excelente com {english_samples:,} amostras")
        else:
            print(f"\n⚠️ DATASET TEM PROBLEMAS MENORES")
            print(f"🔧 Verifique os pontos marcados acima")
            print(f"💡 O treinamento pode ainda assim funcionar")
        
        # Mostrar amostra dos dados
        print(f"\n📋 Amostra dos dados:")
        sample_cols = ['speaker_id', 'text_normalized', 'locale']
        available_cols = [col for col in sample_cols if col in df.columns]
        print(df.head(3)[available_cols].to_string())
        
    except Exception as e:
        print(f"❌ ERRO ao analisar metadata: {e}")
        import traceback
        traceback.print_exc()

else:
    print(f"\n❌ DATASET NÃO DISPONÍVEL")
    print(f"⚠️ Execute os métodos de download mostrados acima")

print(f"\n📍 STATUS FINAL:")
print(f"   Dataset: {'✅' if os.path.exists(dataset_path) else '❌'}")
print(f"   Metadata: {'✅' if os.path.exists(metadata_file) else '❌'}")
print(f"   Áudio: {'✅' if os.path.exists(audio_raw_dir) else '❌'}")
print(f"   Pronto: {'✅ SIM' if os.path.exists(metadata_file) else '❌ NÃO'}")

if os.path.exists(metadata_file):
    print(f"\n🎯 PRÓXIMO PASSO: Execute a célula de configuração do modelo!")
else:
    print(f"\n🎯 PRÓXIMO PASSO: Configure o dataset primeiro!")

In [ ]:
# === CONFIGURAÇÃO DINÂMICA DO MODELO PARA INGLÊS ===
import yaml
import pandas as pd

# Primeiro, verificar o dataset
metadata_file = "data/generated/Dataset-Unificado/metadata.csv"
if os.path.exists(metadata_file):
    df = pd.read_csv(metadata_file)
    total_samples = len(df)
    unique_speakers = df['speaker_id'].nunique()
    locales = df['locale'].unique()
    
    print(f"📊 Dataset carregado:")
    print(f"   📈 Total: {total_samples:,} amostras")
    print(f"   🎤 Falantes: {unique_speakers}")
    print(f"   🌍 Idiomas: {locales}")
    
    # Verificar se é inglês
    if 'en' not in locales:
        print("⚠️ AVISO: Dataset não contém inglês!")
        print(f"   Idiomas encontrados: {locales}")
    else:
        english_samples = len(df[df['locale'] == 'en'])
        print(f"   ✅ Amostras em inglês: {english_samples:,}")
else:
    print("❌ Metadata não encontrada!")
    total_samples = 22910  # Fallback
    unique_speakers = 52

# Usar configuração específica para A100 se disponível
if CONFIG_NAME == "vits2_english_a100_optimized":
    # Carregar configuração A100 otimizada
    config_path = f"configs/training/{CONFIG_NAME}.yaml"
    if os.path.exists(config_path):
        with open(config_path, 'r') as f:
            config = yaml.safe_load(f)
        print("✅ Usando configuração A100 otimizada!")
    else:
        print("⚠️ Configuração A100 não encontrada, criando...")
        config = {}
else:
    config = {}

# Configuração base ou atualização
if not config or 'model' not in config:
    if DEBUG_MODE:
        model_config = {
            "text_encoder_hidden_dim": 128,
            "latent_dim": 128,
            "speaker_embedding_dim": 256,
            "generator_initial_channels": 256,
            "decoder_hidden_dim": 256,
        }
    else:
        # Configuração baseada na GPU
        if "A100" in gpu_name:
            model_config = {
                "text_encoder_hidden_dim": 512,
                "latent_dim": 512,
                "speaker_embedding_dim": 1024,
                "generator_initial_channels": 1024,
                "decoder_hidden_dim": 1024,
            }
        else:
            model_config = {
                "text_encoder_hidden_dim": 192,
                "latent_dim": 192,
                "speaker_embedding_dim": 512,
                "generator_initial_channels": 512,
                "decoder_hidden_dim": 512,
            }
    
    config = {
        "model": {
            "name": "VITS2",
            "mel_channels": 80,
            "n_speakers": unique_speakers,
            "text_processor": "english",
            "inference_only": False,
            **model_config
        }
    }

# Atualizar configurações de treinamento
config.update({
    "training": {
        "learning_rate": 2.0e-4,
        "batch_size": BATCH_SIZE,
        "max_epochs": EPOCHS,
        "accumulate_grad_batches": 1,
        "max_grad_norm": 1.0,
        "mel_loss_weight": 45.0,
        "kl_loss_weight": 1.0,
        "adv_loss_weight": 1.0,
        "fm_loss_weight": 2.0,
        "duration_loss_weight": 1.0,
        "use_amp": True,
        "gradient_clip_val": 1.0,
        "discriminator_update_frequency": 1,
        "scheduler": {
            "name": "ReduceLROnPlateau",
            "mode": "min",
            "factor": 0.5,
            "patience": 15 if not DEBUG_MODE else 5,
            "min_lr": 1.0e-6
        }
    },
    "data": {
        "dataset_format": "valetts",
        "data_dir": "data/generated/Dataset-Unificado",
        "metadata_file": "data/generated/Dataset-Unificado/metadata.csv",
        "language": "en",  # CRÍTICO: usar "en" para inglês
        "locale_column": "locale",
        "text_processor": {
            "class": "EnglishTextPreprocessor",
            "use_phonemes": True,
            "normalize_numbers": True,
            "normalize_whitespace": True,
            "lowercase": True,
            "backend": "espeak",
            "language": "en-us"
        },
        "sample_rate": 22050,
        "n_mels": 80,
        "n_fft": 1024,
        "hop_length": 256,
        "win_length": 1024,
        "num_workers": 8 if "A100" in gpu_name else 4,
        "pin_memory": True,
        "persistent_workers": True,
        "use_augmentation": True,
        "volume_range": [0.9, 1.1],
        "pitch_range": [-1, 1]
    },
    "logging": {
        "log_dir": "logs",
        "experiment_name": f"vits2_english_{CONFIG_NAME.split('_')[-1]}",
        
        # SISTEMA HÍBRIDO DE CHECKPOINTS
        "checkpoint": {
            # Checkpoints principais (2 últimos - se sobrescrevem)
            "dirpath": f"checkpoints/vits2_english_{CONFIG_NAME.split('_')[-1]}",
            "filename": f"vits2_english-{{epoch:03d}}-{{epoch/val_loss_total:.3f}}",
            "monitor": "epoch/val_loss_total",
            "mode": "min",
            "save_top_k": 2,  # Apenas 2 checkpoints recentes
            "save_last": True,
            "every_n_epochs": 1,  # Salvar a cada época
            "auto_insert_metric_name": False,
            "save_on_train_epoch_end": True
        },
        
        # Checkpoints de backup (a cada 10 épocas - permanentes)
        "checkpoint_backup": {
            "enabled": True,
            "dirpath": f"checkpoints/vits2_english_{CONFIG_NAME.split('_')[-1]}/backup",
            "filename": f"vits2_english_backup-{{epoch:03d}}-{{epoch/val_loss_total:.3f}}",
            "every_n_epochs": 10,  # Backup a cada 10 épocas
            "save_top_k": -1,  # Salvar todos os backups (nunca sobrescrever)
            "monitor": "epoch/val_loss_total",
            "mode": "min"
        },
        
        "early_stopping": {
            "monitor": "epoch/val_loss_total",
            "mode": "min",
            "patience": 10 if DEBUG_MODE else 30,
            "min_delta": 0.001
        },
        
        "tensorboard": {
            "save_dir": "logs/tensorboard",
            "name": f"vits2_english_{CONFIG_NAME.split('_')[-1]}"
        }
    },
    "hardware": {
        "accelerator": "gpu",
        "devices": 1,
        "precision": "16-mixed",
        "strategy": "auto",
        "benchmark": True if "A100" in gpu_name else False
    },
    "validation": {
        "val_check_interval": 1.0,
        "generate_samples": True,
        "sample_every_n_epochs": 1,  # Gerar amostras a cada época
        "limit_val_batches": 1.0,
        "num_sanity_val_steps": 2
    },
    "dataset_config": {
        "expected_locale": "en",  # CRÍTICO: valor correto para inglês
        "validate_files": True,
        "cache_preprocessing": True,
        "audio_column": "audio_path",
        "text_column": "text_normalized",
        "speaker_column": "speaker_id",
        "locale_column": "locale"
    },
    
    # CONFIGURAÇÃO LLM INTEGRADA
    "llm_monitor": LLM_CONFIG,
    
    # CONFIGURAÇÃO DE AMOSTRAS DE ÁUDIO
    "audio_sampling": {
        "enabled": True,
        "sample_every_n_epochs": 1,  # Gerar amostras a cada época
        "num_samples": 3,  # 3 amostras por checkpoint
        "max_length": 10.0,  # Máximo 10 segundos
        "sample_rate": 22050,
        "save_dir": "samples",
        "speakers_to_sample": [0, 1, 2]  # Primeiros 3 speakers
    }
})

# Adicionar limitação de amostras para debug
if DEBUG_MODE:
    config["data"]["max_samples_debug"] = MAX_SAMPLES

# Criar diretórios necessários
os.makedirs("configs/training", exist_ok=True)
os.makedirs(config["logging"]["checkpoint"]["dirpath"], exist_ok=True)
os.makedirs(config["logging"]["checkpoint_backup"]["dirpath"], exist_ok=True)  # Diretório backup
os.makedirs("logs/tensorboard", exist_ok=True)
os.makedirs("samples", exist_ok=True)

# Salvar configuração
config_path = f"configs/training/{CONFIG_NAME}.yaml"
with open(config_path, 'w') as f:
    yaml.dump(config, f, default_flow_style=False, indent=2)

print(f"✅ Configuração criada: {config_path}")
print(f"🎯 Modo: {CONFIG_NAME.upper()}")
print(f"📊 Épocas: {EPOCHS}")
print(f"📦 Batch Size: {BATCH_SIZE}")
print(f"🎤 Falantes: {config['model']['n_speakers']}")
print(f"🌍 Idioma: INGLÊS (locale='en')")
print(f"🤖 Monitor LLM: {'✅ Ativo' if LLM_CONFIG['enabled'] else '❌ Desabilitado'}")

print(f"\n💾 Sistema Híbrido de Checkpoints:")
print(f"   📦 Checkpoints recentes: 2 (se sobrescrevem a cada época)")
print(f"   🔒 Checkpoints backup: A cada 10 épocas (permanentes)")
print(f"   📁 Diretório principal: {config['logging']['checkpoint']['dirpath']}")
print(f"   🗄️ Diretório backup: {config['logging']['checkpoint_backup']['dirpath']}")
print(f"   🎵 Amostras de áudio: A cada época")

# Estimar espaço em disco
epochs_to_run = EPOCHS
recent_checkpoints_size = 2 * 150  # 2 checkpoints × ~150MB cada
backup_checkpoints_count = (epochs_to_run // 10) + 1
backup_checkpoints_size = backup_checkpoints_count * 150
total_checkpoints_size = recent_checkpoints_size + backup_checkpoints_size

print(f"\n💾 Estimativa de Espaço em Disco:")
print(f"   📦 Checkpoints recentes: ~{recent_checkpoints_size}MB")
print(f"   🔒 Checkpoints backup: ~{backup_checkpoints_size}MB ({backup_checkpoints_count} arquivos)")
print(f"   🎵 Amostras de áudio: ~{epochs_to_run * 3 * 0.03:.1f}MB")
print(f"   📊 Total estimado: ~{total_checkpoints_size + (epochs_to_run * 3 * 0.03):.0f}MB")

print(f"\n💾 Dimensões do modelo:")
print(f"   - Hidden: {config['model']['text_encoder_hidden_dim']}")
print(f"   - Speaker: {config['model']['speaker_embedding_dim']}")
print(f"   - Generator: {config['model']['generator_initial_channels']}")
print(f"   - Decoder: {config['model']['decoder_hidden_dim']}")

# Verificar compatibilidade de dimensões CRÍTICA
if config['model']['speaker_embedding_dim'] == config['model']['decoder_hidden_dim']:
    print("✅ Dimensões compatíveis - Sem erro de tensor!")
else:
    print("❌ AVISO: Dimensões incompatíveis detectadas!")
    print(f"   Speaker: {config['model']['speaker_embedding_dim']}")
    print(f"   Decoder: {config['model']['decoder_hidden_dim']}")

print(f"\n🚀 Configuração otimizada para {gpu_name} pronta!")

if LLM_CONFIG['enabled']:
    print("\n🤖 Monitor LLM ativo - benefícios:")
    print("   📊 Análise inteligente do progresso")
    print("   ⚙️ Ajustes automáticos de hiperparâmetros")
    print("   🎯 Detecção precoce de problemas")
    print("   📈 Relatórios detalhados de treinamento")

In [ ]:
# === INICIALIZAÇÃO DO TENSORBOARD ===
# Configurar TensorBoard antes do treinamento para monitoramento em tempo real

# Verificar se as variáveis necessárias estão definidas
try:
    config
    CONFIG_NAME
except NameError as e:
    print(f"❌ ERRO: Execute a célula de configuração do modelo primeiro!")
    raise SystemExit("Execute as células anteriores primeiro!")

print("📊 Configurando TensorBoard para monitoramento em tempo real...")

# Configurar diretório do TensorBoard
tensorboard_dir = config["logging"]["tensorboard"]["save_dir"]
experiment_name = config["logging"]["tensorboard"]["name"]

print(f"📁 Diretório: {tensorboard_dir}")
print(f"🎯 Experimento: {experiment_name}")

# Criar diretório se não existir
os.makedirs(tensorboard_dir, exist_ok=True)

# Inicializar TensorBoard no Colab
try:
    # Carregar extensão do TensorBoard
    %load_ext tensorboard
    
    # Iniciar TensorBoard
    %tensorboard --logdir={tensorboard_dir} --reload_interval=30
    
    print("✅ TensorBoard iniciado com sucesso!")
    print("📊 Acesse a aba 'TensorBoard' acima para ver os gráficos em tempo real")
    print(f"🔄 Atualização automática a cada 30 segundos")
    print("\n📈 Métricas disponíveis:")
    print("   - Loss total de treinamento e validação")
    print("   - Loss MEL, KL, Adversarial, Feature Matching")
    print("   - Learning rate e gradient norm")
    print("   - Amostras de áudio geradas")
    print("   - Gráficos de convergência")
    
    if LLM_CONFIG['enabled']:
        print("   - Análises do Monitor LLM (a cada 10 épocas)")
    
except Exception as e:
    print(f"⚠️ Erro ao inicializar TensorBoard: {e}")
    print("💡 TensorBoard será inicializado automaticamente durante o treinamento")

print("\n🎯 Sistema pronto para treinamento com monitoramento completo!")

In [ ]:
# === INICIAR TREINAMENTO OTIMIZADO PARA INGLÊS ===

# Verificar se as variáveis necessárias estão definidas
try:
    CONFIG_NAME
    config_path
    gpu_name
    gpu_memory
    BATCH_SIZE
    ESTIMATED_TIME
    EXPECTED_PERFORMANCE
    LLM_CONFIG
except NameError as e:
    print(f"❌ ERRO: Variável não definida - {e}")
    print("💡 SOLUÇÃO: Execute as células anteriores em ordem:")
    print("   1. Configuração da API OpenRouter")
    print("   2. Configuração do modo de treinamento")
    print("   3. Setup do sistema")
    print("   4. Instalação de dependências")
    print("   5. Download do dataset")
    print("   6. Configuração do modelo")
    print("   7. Esta célula de treinamento")
    print("")
    print("🔄 Execute novamente a partir da célula 2 (Configuração do modo)")
    raise SystemExit("Execute as células anteriores primeiro!")

print(f"🚀 Iniciando treinamento VITS2 - Modo: {CONFIG_NAME.upper()}")
print(f"📁 Configuração: {config_path}")
print(f"🎮 GPU: {gpu_name} ({gpu_memory} MB)")
print(f"📦 Batch Size: {BATCH_SIZE}")
print(f"⏱️ Estimativa: {ESTIMATED_TIME}")
print(f"⚡ Performance esperada: {EXPECTED_PERFORMANCE}")
print(f"🤖 Monitor LLM: {'✅ Ativo' if LLM_CONFIG['enabled'] else '❌ Desabilitado'}")
print("\n" + "="*60)

# Verificar se temos script específico para inglês
english_script = "scripts/train_vits2_english.py"
if os.path.exists(english_script):
    print("✅ Usando script específico para inglês")
    base_cmd = f"python {english_script} --config {config_path}"
else:
    print("⚠️ Script específico não encontrado, usando script padrão")
    base_cmd = f"python scripts/train_vits2.py --config {config_path}"

# Configurar comando baseado no LLM
if LLM_CONFIG['enabled']:
    print("🤖 Monitor LLM ativo - treinamento inteligente habilitado")
    cmd = base_cmd  # LLM já está na configuração
else:
    print("🔧 Monitor LLM desabilitado - usando modo padrão")
    cmd = f"{base_cmd} --disable-llm"

# Configurar CUDA para otimização máxima
os.environ['CUDA_LAUNCH_BLOCKING'] = '0'  # Async CUDA
os.environ['TORCH_CUDNN_V8_API_ENABLED'] = '1'  # CuDNN v8
os.environ['TORCH_ALLOW_TF32_CUBLAS_OVERRIDE'] = '1'  # TF32 para A100

if "A100" in gpu_name:
    print("🚀 Configurações especiais para A100:")
    print("   - TF32 habilitado para máxima performance")
    print("   - CuDNN v8 API ativado")
    print("   - Tensor cores ativados automaticamente")
    os.environ['TORCH_ALLOW_TF32'] = '1'
    os.environ['TORCH_CUDNN_ALLOW_TF32'] = '1'

# Configurações específicas para Monitor LLM
if LLM_CONFIG['enabled']:
    print("\n🤖 Configurações do Monitor LLM:")
    print(f"   🧠 Modelo: {LLM_CONFIG['model']}")
    print(f"   📊 Análise a cada: {LLM_CONFIG['monitor_every_epochs']} épocas")
    print(f"   🔄 Provider: {LLM_CONFIG['provider']}")
    print("   📈 Funcionalidades ativas:")
    print("      - Otimização automática de learning rate")
    print("      - Detecção de problemas de convergência")
    print("      - Sugestões de ajustes de hiperparâmetros")
    print("      - Relatórios detalhados de progresso")

# Executar treinamento com output em tempo real
print(f"\n📝 Comando: {cmd}")
print("🏁 Iniciando treinamento...\n")

try:
    # Executar treinamento
    run_command(cmd, f"Treinamento VITS2 {CONFIG_NAME.upper()}")
    
    print("\n" + "="*60)
    print("🎉 TREINAMENTO CONCLUÍDO COM SUCESSO!")
    
    # Verificar checkpoints gerados
    checkpoint_dir = config["logging"]["checkpoint"]["dirpath"]
    if os.path.exists(checkpoint_dir):
        checkpoints = [f for f in os.listdir(checkpoint_dir) if f.endswith('.ckpt')]
        print(f"📁 Checkpoints gerados: {len(checkpoints)}")
        
        if checkpoints:
            # Mostrar checkpoints mais recentes
            checkpoint_paths = [os.path.join(checkpoint_dir, f) for f in checkpoints]
            checkpoint_paths.sort(key=os.path.getmtime, reverse=True)
            
            print("📦 Checkpoints disponíveis:")
            for i, ckpt in enumerate(checkpoint_paths[:3]):  # Mostrar 3 mais recentes
                size_mb = os.path.getsize(ckpt) / 1024 / 1024
                print(f"   {i+1}. {os.path.basename(ckpt)} ({size_mb:.1f} MB)")
    
    # Verificar logs do TensorBoard
    tensorboard_dir = config["logging"]["tensorboard"]["save_dir"]
    if os.path.exists(tensorboard_dir):
        print(f"📊 Logs TensorBoard: {tensorboard_dir}")
    
    # Verificar logs do Monitor LLM se ativo
    if LLM_CONFIG['enabled']:
        llm_logs_dir = "logs/llm_monitor"
        if os.path.exists(llm_logs_dir):
            llm_files = [f for f in os.listdir(llm_logs_dir) if f.endswith('.json')]
            print(f"🤖 Análises LLM geradas: {len(llm_files)}")
            if llm_files:
                latest_analysis = sorted(llm_files)[-1]
                print(f"   📄 Última análise: {latest_analysis}")
    
    print(f"\n✅ Modelo treinado com sucesso para inglês!")
    print(f"🎯 Configuração: {CONFIG_NAME}")
    print(f"🌍 Idioma: Inglês")
    print(f"📈 Dataset: 22.910 amostras")
    print(f"🤖 Monitor LLM: {'Usado' if LLM_CONFIG['enabled'] else 'Não usado'}")

except Exception as e:
    print(f"\n❌ ERRO DURANTE TREINAMENTO:")
    print(f"   {str(e)}")
    print(f"\n🔍 Diagnóstico:")
    
    # Verificar se arquivos necessários existem
    required_files = [
        config_path,
        config["data"]["metadata_file"],
        config["data"]["data_dir"]
    ]
    
    for file_path in required_files:
        if os.path.exists(file_path):
            print(f"   ✅ {file_path}")
        else:
            print(f"   ❌ {file_path} - NÃO ENCONTRADO")
    
    # Verificar GPU
    try:
        import torch
        if torch.cuda.is_available():
            print(f"   ✅ CUDA disponível: {torch.cuda.get_device_name(0)}")
        else:
            print("   ❌ CUDA não disponível")
    except:
        print("   ❌ PyTorch não instalado ou erro CUDA")
    
    # Verificar configuração LLM se ativa
    if LLM_CONFIG['enabled']:
        if os.environ.get('OPENROUTER_API_KEY'):
            print("   ✅ API Key OpenRouter configurada")
        else:
            print("   ❌ API Key OpenRouter não encontrada")
    
    raise

In [ ]:
# === VISUALIZAÇÃO DE AMOSTRAS GERADAS ===
# Reproduzir e analisar amostras de áudio geradas durante o treinamento

import glob
import os
from IPython.display import Audio, display, HTML

# Verificar se existem amostras geradas
samples_dir = "samples"
if not os.path.exists(samples_dir):
    print("📁 Nenhuma amostra encontrada ainda")
    print("💡 As amostras serão geradas automaticamente durante o treinamento")
else:
    # Encontrar todas as amostras por época
    sample_files = glob.glob(f"{samples_dir}/**/*.wav", recursive=True)
    
    if not sample_files:
        print("🎵 Nenhuma amostra de áudio encontrada ainda")
        print("⏳ Aguarde o treinamento gerar as primeiras amostras")
    else:
        print(f"🎵 Encontradas {len(sample_files)} amostras de áudio!")
        
        # Organizar por época
        samples_by_epoch = {}
        for sample_file in sample_files:
            # Extrair número da época do nome do arquivo
            basename = os.path.basename(sample_file)
            if "epoch_" in basename:
                try:
                    epoch_num = int(basename.split("epoch_")[1].split("_")[0])
                    if epoch_num not in samples_by_epoch:
                        samples_by_epoch[epoch_num] = []
                    samples_by_epoch[epoch_num].append(sample_file)
                except:
                    pass
        
        if samples_by_epoch:
            print(f"📊 Amostras organizadas por {len(samples_by_epoch)} épocas")
            
            # Mostrar amostras das últimas 3 épocas
            recent_epochs = sorted(samples_by_epoch.keys())[-3:]
            
            for epoch in recent_epochs:
                print(f"\n🎯 Época {epoch:03d}:")
                epoch_samples = samples_by_epoch[epoch]
                
                for i, sample_file in enumerate(epoch_samples[:3]):  # Máximo 3 por época
                    print(f"   🎵 Amostra {i+1}:")
                    
                    # Extrair informações do nome do arquivo
                    basename = os.path.basename(sample_file)
                    if "speaker_" in basename:
                        speaker_id = basename.split("speaker_")[1].split("_")[0]
                        print(f"      👤 Speaker: {speaker_id}")
                    
                    # Reproduzir áudio
                    try:
                        display(Audio(sample_file, rate=22050))
                        
                        # Mostrar tamanho do arquivo
                        size_kb = os.path.getsize(sample_file) / 1024
                        print(f"      📦 Tamanho: {size_kb:.1f} KB")
                        
                    except Exception as e:
                        print(f"      ❌ Erro ao carregar: {e}")
            
            # Estatísticas gerais
            print(f"\n📈 Estatísticas das Amostras:")
            print(f"   🎵 Total de amostras: {len(sample_files)}")
            print(f"   📊 Épocas com amostras: {len(samples_by_epoch)}")
            
            # Tamanho total
            total_size = sum(os.path.getsize(f) for f in sample_files)
            print(f"   💾 Tamanho total: {total_size / 1024 / 1024:.1f} MB")
            
            # Última época
            if recent_epochs:
                last_epoch = max(recent_epochs)
                print(f"   🎯 Última época: {last_epoch}")
        
        else:
            print("⚠️ Amostras encontradas mas não organizadas por época")
            print("💡 Verifique o formato dos nomes dos arquivos")

# Widget para navegação (se houver muitas amostras)
if 'samples_by_epoch' in locals() and len(samples_by_epoch) > 3:
    print(f"\n🎛️ Use o código abaixo para navegar pelas amostras de épocas específicas:")
    print(f"```python")
    print(f"# Escolha uma época específica")
    print(f"epoch_to_play = 10  # Substitua pelo número da época")
    print(f"if epoch_to_play in samples_by_epoch:")
    print(f"    for sample in samples_by_epoch[epoch_to_play]:")
    print(f"        display(Audio(sample, rate=22050))")
    print(f"```")

print("\n🎯 Esta célula será atualizada automaticamente conforme novas amostras são geradas!")

In [ ]:
# === DOWNLOAD DOS RESULTADOS ===
import zipfile
from datetime import datetime
import glob

# Verificar se as variáveis necessárias estão definidas
try:
    CONFIG_NAME
    config
except NameError as e:
    print(f"❌ ERRO: Variável não definida - {e}")
    print("💡 Execute as células anteriores primeiro!")
    print("   Especialmente a célula de 'Configuração do modelo'")
    raise SystemExit("Execute as células anteriores primeiro!")

# Criar arquivo ZIP com resultados
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
zip_filename = f"valetts_vits2_english_{CONFIG_NAME.split('_')[-1]}_{timestamp}.zip"

print(f"📦 Criando arquivo ZIP: {zip_filename}")
print("📋 Incluindo todos os tipos de checkpoints...")

with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    
    # Adicionar checkpoints principais (2 mais recentes)
    checkpoint_dir = config["logging"]["checkpoint"]["dirpath"]
    recent_checkpoints = []
    if os.path.exists(checkpoint_dir):
        recent_ckpts = glob.glob(f"{checkpoint_dir}/*.ckpt")
        if recent_ckpts:
            recent_checkpoints = sorted(recent_ckpts, key=os.path.getmtime)[-2:]  # 2 mais recentes
            for ckpt in recent_checkpoints:
                zipf.write(ckpt, os.path.relpath(ckpt, '.'))
            print(f"   📦 Checkpoints recentes: {len(recent_checkpoints)} adicionados")
    
    # Adicionar checkpoints de backup (a cada 10 épocas)
    backup_dir = config["logging"]["checkpoint_backup"]["dirpath"]
    backup_checkpoints = []
    if os.path.exists(backup_dir):
        backup_ckpts = glob.glob(f"{backup_dir}/*.ckpt")
        if backup_ckpts:
            backup_checkpoints = sorted(backup_ckpts, key=os.path.getmtime)
            for ckpt in backup_checkpoints:
                zipf.write(ckpt, os.path.relpath(ckpt, '.'))
            print(f"   🔒 Checkpoints backup: {len(backup_checkpoints)} adicionados")

    # Adicionar configuração
    try:
        config_path
        zipf.write(config_path, config_path)
        print(f"   ⚙️ Configuração: {config_path} adicionada")
    except NameError:
        print("   ⚠️ config_path não definido - configuração não incluída")

    # Adicionar amostras geradas
    samples_added = 0
    if os.path.exists("samples"):
        for root, dirs, files in os.walk("samples"):
            for file in files:
                if file.endswith('.wav'):
                    file_path = os.path.join(root, file)
                    zipf.write(file_path, os.path.relpath(file_path, '.'))
                    samples_added += 1
        print(f"   🎵 Amostras de áudio: {samples_added} adicionadas")

    # Adicionar logs principais
    logs_added = 0
    log_files = ["logs/training.log", "logs/tensorboard"]
    for log_item in log_files:
        if os.path.exists(log_item):
            if os.path.isfile(log_item):
                zipf.write(log_item, log_item)
                logs_added += 1
            elif os.path.isdir(log_item):
                for root, dirs, files in os.walk(log_item):
                    for file in files:
                        if not file.startswith('.'):  # Ignorar arquivos ocultos
                            file_path = os.path.join(root, file)
                            zipf.write(file_path, os.path.relpath(file_path, '.'))
                            logs_added += 1
    if logs_added > 0:
        print(f"   📊 Logs: {logs_added} arquivos adicionados")

print(f"\n✅ Arquivo criado: {zip_filename}")
print(f"💾 Tamanho: {os.path.getsize(zip_filename) / 1024 / 1024:.1f} MB")

# Estatísticas detalhadas dos checkpoints
print(f"\n📊 Resumo dos Checkpoints:")
if recent_checkpoints:
    print(f"   📦 Checkpoints Recentes ({len(recent_checkpoints)}):")
    for i, ckpt in enumerate(recent_checkpoints):
        size_mb = os.path.getsize(ckpt) / 1024 / 1024
        epoch = "???"
        try:
            # Extrair época do nome do arquivo
            import re
            match = re.search(r'epoch_(\d+)', os.path.basename(ckpt))
            if not match:
                match = re.search(r'-(\d+)-', os.path.basename(ckpt))
            if match:
                epoch = match.group(1)
        except:
            pass
        print(f"      {i+1}. Época {epoch} - {size_mb:.1f}MB")

if backup_checkpoints:
    print(f"   🔒 Checkpoints Backup ({len(backup_checkpoints)}):")
    for i, ckpt in enumerate(backup_checkpoints):
        size_mb = os.path.getsize(ckpt) / 1024 / 1024
        epoch = "???"
        try:
            import re
            match = re.search(r'epoch_(\d+)', os.path.basename(ckpt))
            if not match:
                match = re.search(r'-(\d+)-', os.path.basename(ckpt))
            if match:
                epoch = match.group(1)
        except:
            pass
        print(f"      {i+1}. Época {epoch} - {size_mb:.1f}MB")

# Download no Colab
try:
    if 'google.colab' in sys.modules:
        from google.colab import files
        print("\n⬇️ Iniciando download...")
        files.download(zip_filename)
        print("✅ Download concluído!")
except:
    print("\nℹ️ Não está no Colab ou erro no download - arquivo salvo localmente")

print("\n🎯 TREINAMENTO FINALIZADO!")
try:
    print(f"📊 Modo: {CONFIG_NAME.upper()}")
    try:
        DEBUG_MODE
        print(f"⏱️ Status: {'Teste concluído' if DEBUG_MODE else 'Produção concluída'}")
    except NameError:
        print("⏱️ Status: Concluído")
except NameError:
    print("📊 Modo: Não definido")

print(f"📦 Resultados salvos em: {zip_filename}")
print(f"\n💾 Sistema Híbrido de Checkpoints implementado:")
print(f"   📦 Checkpoints recentes: {len(recent_checkpoints) if recent_checkpoints else 0} (últimos 2)")
print(f"   🔒 Checkpoints backup: {len(backup_checkpoints) if backup_checkpoints else 0} (a cada 10 épocas)")
print(f"   🎵 Amostras de áudio: {samples_added} arquivos")

# Verificar espaço total usado
try:
    total_recent = sum(os.path.getsize(f) for f in recent_checkpoints) / 1024 / 1024 if recent_checkpoints else 0
    total_backup = sum(os.path.getsize(f) for f in backup_checkpoints) / 1024 / 1024 if backup_checkpoints else 0
    print(f"\n💾 Espaço em disco utilizado:")
    print(f"   📦 Checkpoints recentes: {total_recent:.1f}MB")
    print(f"   🔒 Checkpoints backup: {total_backup:.1f}MB")
    print(f"   📊 Total checkpoints: {total_recent + total_backup:.1f}MB")
except:
    print("\n⚠️ Não foi possível calcular espaço em disco")